# Astronomy 8824 - Problem Set 7

The goal of this problem set is to gain familiarity with systematics and nuisance parameters.

This problem set was developed by David Weinberg, with some modifications by Paul Martini.

**Note:** Parts 2 and 3 of this problem set illustrate two _different_ ways of dealing with calibration errors and their impact on a measurement of $H_0$. You'll compare results at the end, but don't automatically carry your ideas from Problem 2 over to Problem 3.

$\newcommand{\expect}[1]{{\left\langle #1 \right\rangle}}$

In [1]:
import numpy as np
from numpy import matrix
from numpy import linalg
import matplotlib.pyplot as plt
from matplotlib.gridspec import GridSpec
from astropy.io import ascii
import math
import corner

# matplotlib settings 
SMALL_SIZE = 14
MEDIUM_SIZE = 16
BIGGER_SIZE = 18

plt.rc('font', size=SMALL_SIZE)          # controls default text sizes
plt.rc('axes', titlesize=SMALL_SIZE)     # fontsize of the axes title
plt.rc('axes', labelsize=BIGGER_SIZE)    # fontsize of the x and y labels
plt.rc('lines', linewidth=2)
plt.rc('axes', linewidth=2)
plt.rc('xtick', labelsize=MEDIUM_SIZE)    # fontsize of the tick labels
plt.rc('ytick', labelsize=MEDIUM_SIZE)    # fontsize of the tick labels
plt.rc('legend', fontsize=MEDIUM_SIZE)    # legend fontsize
plt.rc('figure', titlesize=BIGGER_SIZE)   # fontsize of the figure title

In [2]:
### Some code you may want to use for plotting results in part 3

def gaussian(x, mu, sig):
    '''
    Calculate a gaussian with mean mu and dispersion sig at input points x
    '''
    return np.exp( -0.5 * np.power( (x - mu)/sig, 2.)) / (np.sqrt(2*np.pi)*sig)

def PlotTwoDist(xy1, label1, xy2=None, label2=None, dims=None, addgauss=False, gxsig=False, gysig=False, connect=20): 
    '''
    xy1, xy2: (x,y) points for the two distributions
    label1, label2: labels for the two distributions
    dims: [xc, yc, dx, dy] where xc,yc are the plot center and dx, dy are the half sizes of the plot
    addgauss: (bool) True to overplot Gaussian on each histogram
    gxsig, gysig: (float) sigma values for the two histograms
    '''
    
    if dims is not None:
        xc = dims[0]
        yc = dims[1]
        dx = dims[2]
        dy = dims[3]
        xbins = np.arange(xc - dx, xc + dx, 0.05*dx)
        ybins = np.arange(yc - dy, yc + dy, 0.05*dy)
    else: 
        xbins = []
        ybins = []

    fig = plt.figure(figsize=(8,8))
    gs = GridSpec(4,4)
    ax_scatter = fig.add_subplot(gs[1:4, 0:3])
    ax_xhist = fig.add_subplot(gs[0,0:3])
    ax_yhist = fig.add_subplot(gs[1:4,3])
    ax_scatter.scatter(xy1[0], xy1[1], color='k', s=1, label=label1)
    ax_xhist.hist(xy1[0], bins=xbins, histtype='step', color='k', density=True)
    ax_yhist.hist(xy1[1], bins=ybins, histtype='step', color='k', density=True, orientation='horizontal')

    if xy2 is not None: 
        ax_scatter.scatter(xy2[0], xy2[1], color='r', s=1, label=label2)
        ax_scatter.plot(xy2[0][:connect], xy2[1][:connect], color='r', ls='-')
        ax_xhist.hist(xy2[0], bins=xbins, histtype='step', color='r', density=True)
        ax_xhist.hist(xy2[0], bins=xbins, histtype='step', color='r', density=True)
        ax_yhist.hist(xy2[1], bins=xbins, histtype='step', color='r', density=True, orientation='horizontal')

    if addgauss and gxsig and gysig: 
        if dims is not None:
            ggxx = np.linspace(xc - dx, xc + dx, 100)
            ggx = gaussian(ggxx, xc, gxsig)
            ax_xhist.plot(ggxx, ggx)
            ggyy = np.linspace(yc - dy, yc + dy, 100)
            ggy = gaussian(ggyy, yc, gysig)
            ax_yhist.plot(ggy, ggyy)            
        else: 
            gg = np.linspace(-1*size, size, 100)
            ggx = gaussian(gg, 0., gxsig)
            ggy = gaussian(gg, 0., gysig)
            ax_xhist.plot(gg, ggx)
            ax_yhist.plot(ggy, gg)
    
    plt.setp(ax_xhist.get_xticklabels(), visible=False)
    plt.setp(ax_xhist.get_yticklabels(), visible=False)
    plt.setp(ax_yhist.get_xticklabels(), visible=False)
    plt.setp(ax_yhist.get_yticklabels(), visible=False)
    ax_scatter.set_xlabel("X", fontsize=16)
    ax_scatter.set_ylabel("Y", fontsize=16)
    if dims is not None: 
        ax_scatter.set_xlim(xc - dx, xc + dx)
        ax_scatter.set_ylim(yc - dy, yc + dy)
        ax_xhist.set_xlim(xc - dx, xc + dx)
        ax_yhist.set_ylim(yc - dy, yc + dy)


    ax_xhist.set_ylabel("N", fontsize=16)
    ax_yhist.set_xlabel("N", fontsize=16)
    ax_scatter.legend(frameon=False, fontsize=16)

### 1. Best-fit slope and intercept with correlated errors

Note the Homework/data/ subdirectory within this repository. This contains ten data sets generated with five different covariance matrices, with two random number seeds per covariance matrix. The five covariance matrices are the ones from Part 4 of Problem Set 5. (There is also a data file h0.dat for Part 3.)

#### a) Write an analytic expression for the best-fit intercept $\theta_1$ and best-fit slope $\theta_2$ in terms of the Fisher matrix and the covariance matrix.

#### Answer

#### b) For each of these ten realizations compute the best-fit slope and intercept, using the appropriate covariance matrix for each case. Also report the uncertainties on both quantities.

In [3]:
#### Answer: 

### 2. Calibration errors in H0 measurement, treated via error covariance

The goal of this exercise is to estimate the Hubble constant using Type Ia supernova distances to galaxies and quantify the uncertainties.

Assume (unrealistically) that you have calibrated the mean absolute magnitude (at peak luminosity) of Type Ia SNe with no uncertainty, from local galaxies whose distance is known by other means but which are too close to estimate $H_0$ because of peculiar velocities.

By comparing the peak apparent magnitude of SNe found in distant galaxies to this absolute magnitude, you get an estimate of $\ln d$ to each of these galaxies. Assume that the error in $\ln d$ has a constant value $\sigma$ for all of your sample galaxies, which may be realistic if the error is dominated by the intrinsic scatter of supernova luminosities rather than by measurement uncertainties. Note that we are using $\ln d$ rather than $d$ because a realistic error distribution is closer to Gaussian in $\ln d$.

You also measure the recession velocity $v$ for each galaxy, with negligible uncertainty (which requires your galaxies to be distant enough that peculiar velocities can be ignored).

Hubble’s law, $v = H_0 d$, can thus be written $\ln d = \ln v \, - \, \ln H_0$. If we think of the velocities as our independent variables $x_i$ and the distance measurements as our data values $y_i$, then inferring $H_0$ comes down to determining the intercept of $y = x+b$, where the slope is fixed to unity because we are assuming that Hubble’s law is correct for some value of $H_0$.

#### a. No calibration errors. 

For 16 measurements each with $\sigma = 0.08$, what is the expected fractional uncertainty in $H_0$?

#### Answer

#### b. Global calibration uncertainty

Now throw in a realistic wrinkle: the distant supernovae are observed with a different telescope and filter set from the local calibrator sample, so there is an uncertainty in $\ln d$ that affects all of the measurements in the same way. Specifically, if the calibration error is $\Delta$, then the observed value $y_{i,obs}$ will be Gaussian distributed with dispersion $\sigma$ about $y_{i,true} + \Delta$, where $y_i = \ln d_i$.

We don’t know $\Delta$, of course, or we would just remove it and calibrate our data to the same system. However, we may know the plausible range of $\Delta$ — i.e., the calibration uncertainty $\sigma^2_\Delta = <\Delta^2>$. (We’ve done the best we can on calibration, so $<\Delta>= 0$.)

The value of $\sigma_\Delta$ is just about half the uncertainty of the photometric calibration in magnitudes. **Why is this the case?** 

A realistic value for good observations might be $\sigma_\Delta \approx 0.01 - 0.02$. 

Give a mathematical argument that the covariance matrix of the errors 
in this case is

$$
C_{ij} = \sigma^2\delta_{ij} + \sigma_{\Delta}^2,
$$

where $\delta_{ij}$ is the Kronecker-delta. (Hint: go back to the basic definition of $C_{ij}$, and think
about what happens when you take expectation values.)

#### Answer

#### c. For N = 16, $\sigma = 0.08, \sigma_\Delta = 0.02$, what is the uncertainty in $H_0$?

Provide a numerical result.

In [4]:
#### Answer 

#### d. For what conditions on N, $\sigma$, and $\sigma_\Delta$  does the calibration uncertainty make an important contribution to the overall uncertainty in $H_0$?

#### Answer

#### e. Suppose the sample comes from 2 different telescopes with different calibration uncertainties, that is sample one is points 1-8 and has $\sigma_{\Delta,1}$ and sample two is points 9-16 and has $\sigma_{\Delta,2}$.  Assume the two calibration uncertainties are uncorrelated. What is the covariance matrix in this case? 

You may describe its general structure. You do not need to provide the matrix. 

#### Answer

#### f. What is the uncertainty in $H_0$ for $\sigma_{\Delta,1} = \sigma_{\Delta,2} = 0.02$? For $\sigma_{\Delta,1} = \sigma_{\Delta,2} = 0.04$? For $\sigma_{\Delta,1} = \sigma_{\Delta,2} = 0.01$? 

In [5]:
#### Answer

### 3. Calibration errors in $H_0$ treated via marginalization

The dataset h0.data is also in the data/ subdirectory. This file has 16 entries, and columns two and thre are $\ln v$ in km/s and $\ln d$ in Mpc. 

Data points $1-8$ come from Telescope 1 with calibration uncertainty $\Delta_1$ and points $9-16$ from
Telescope 2 with calibration uncertainty $\Delta_2$.

Assume that apart from the calibration uncertainty the errors $\sigma$ in $\ln d$ are Gaussian with dispersion 0.08.

Treat $\Delta_1$ and $\Delta_2$ as nuisance parameters, and adopt Gaussian priors on their values: $p(\Delta) = (2\pi \sigma^2_\Delta)^{-1/2}\exp(-\Delta^2/2\sigma^2_\Delta)$ with $\sigma_\Delta = 0.02$ 
for both calibration offsets.

Adopt a flat prior on $\ln H_0$. 

With this flat prior, one can write the posterior probabililty of the data and the calibration offsets as:

$$
p(\ln H_0,\Delta_1,\Delta_2) \propto 
p(\Delta_1)p(\Delta_2)p({\rm Data}| \ln H_0,\Delta_1,\Delta_2) 
\propto \exp(-\chi^2/2)
$$

#### a) What is the expression for $\chi^2$?

#### Answer 

#### b. MCMC Analysis 

Write an MCMC program for the 3-dimensional parameter space $\ln H_0$, $\Delta_1$, $\Delta_2$, using the data points above.

From your MCMC chain, plot distributions in the three parameter planes $\ln H_0$ vs. $\Delta_1$, $\ln H_0$ vs. $\Delta_2$, $\Delta_1$ vs. $\Delta_2$.

In [6]:
#### Answer 

#### c. What is your estimate of $H_0$ and its fractional error, marginalized over $\Delta_1$ and  $\Delta_2$?

And what can you infer from your data about the _relative_ values of the calibration errors $\Delta_1$ and $\Delta_2$, specifically of $\Delta_1-\Delta_2$?

#### Answer

#### d) If you widen the prior on the calibrations to $\sigma_\Delta = 0.04$, how does the fractional error in $H_0$ change? 

#### Answer

#### e) What if you sharpen it to $\sigma_\Delta = 0.01$?

#### Answer

#### e.  How do the uncertainties in $H_0$ that you find from this marginalization approach compare to the ones you computed via the covariance matrix approach in Part 2?

#### Answer